# CIFAR-10 Example

- **Instructor**: Jongwoo Lim / Jiun Bae
- **Email**: [jlim@hanyang.ac.kr](mailto:jlim@hanyang.ac.kr) / [jiunbae.623@gmail.com](mailto:jiunbae.623@gmail.com)

## CIFAR dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

### Import packages

First of all, Import some packages for using PyTorch.

- torch.nn: The **Network** of PyTorch basically starts with nn.Module.
- torch.nn.functional: for **Functions** such as *ReLU*, *MaxPool* (in this example)
- torch.optim: for **Optimizers**
- torchvision: Handling **Datasets**

Numpy the basic scientific computing package used in customary.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Dataset

PyTorch basically provides CIFAR-10 Dataset and support download in running code!

In [ ]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer',
		   'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
DATASET_DIR = '../data' # path to download cifar-10 dataset

TRAIN_DATASET = datasets.CIFAR10(DATASET_DIR,   # Dataset root path
                                 train=True,     # Train data
                                 download=True)  # Download if not exist

TEST_DATASET = datasets.CIFAR10(DATASET_DIR,    # Dataset root path
                                train=False)     # Test data

#### Network

This is a simple convolution layer network includes 3 conv layer and 2 fc layer.

In [ ]:
class ConvNetwork(nn.Module):
    """Simple Neural Network contains conv layer and fc layer
    """
    def __init__(self):
        super(ConvNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(50*5*5, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 50*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Define Train and Test functions

In [ ]:
from typing import Tuple

def train(model, device, train_loader, optimizer, criterion) -> float:
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    return loss.item()


def test(model, device, test_loader, criterion) -> Tuple[float, float, torch.Tensor]:
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, accuracy, output

In [ ]:
torch.manual_seed(42) # 42, THE ANSWER TO LIFE, THE UNIVERSE AND EVERYTHING

batch = 64            # batch size
lr = .01              # learning rate
epochs = 10

TRAIN_DATASET.transform = transforms.ToTensor()
train_loader = torch.utils.data.DataLoader(TRAIN_DATASET,
                                           batch_size=64,
                                           shuffle=True)

TEST_DATASET.transform = transforms.ToTensor()
test_loader = torch.utils.data.DataLoader(TEST_DATASET,
                                          batch_size=64,
                                          shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = ConvNetwork().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = F.nll_loss

In [ ]:
for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    test_loss, accuracy, _ = test(model, device, test_loader, criterion)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "cifar_cnn.pt")

## Change Optimizer

In [ ]:
model = ConvNetwork().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = F.nll_loss

In [ ]:
for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    test_loss, accuracy, _ = test(model, device, test_loader, criterion)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "cifar_cnn.pt")

## Classwise accuracy

Accuracy per class

In [ ]:
from PIL import Image
from IPython.display import display

def show(ary):
    display(Image.fromarray(ary))

In [ ]:
model.eval()
correct = np.zeros(10)
total = np.zeros(10)
images = np.zeros((10, 3, 32, 32))

for index, (image, label) in enumerate(TEST_DATASET):
    total[label] += 1
    
    pred = model(image.to(device).unsqueeze(0))
    pred = pred.argmax(dim=1).squeeze()
    if label == pred:
        if not correct[label]:
            images[label] = np.array(image).copy()
        correct[label] += 1

In [ ]:
for index, (image, acc) in enumerate(zip(images, correct / total)):
    show(np.transpose((image * 255).astype(np.uint8), (1, 2, 0)))
    print(f'{CLASSES[index]}: {acc * 100:.2f}')

# Q1. Change learning rate

How does the learning rate affect the model's learning?

- Change Adam optimizer learning rate 0.01 -> 0.001

How can we define the learning rate?

- There is no fixed answer.

In [ ]:
model = ConvNetwork().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr*.1)
criterion = F.nll_loss

In [ ]:
for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    test_loss, accuracy, _ = test(model, device, test_loader, criterion)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "cifar_cnn.pt")

# Q2. Change criterion

Criterion(objective function) aka loss(or error) function is a function to be minimized(or maximized).

The result of that function is called `loss`, `error`, `cost` or `penalty`.


In this time we using `F.nll_loss`(*negative log-likelihood loss*).

$$loss(x, class) = -log (\frac{exp(x[class])} {\sum_j exp(x[j])}) = -x[class] + log(\sum_j exp(x[j]))$$

We using categorical(one-hot encoded) label. So, using categorical loss.

- Replace `F.nll_loss` to `F.cross_entropy`

In [ ]:
model = ConvNetwork().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr*.1)
criterion = F.cross_entropy

In [ ]:
for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    test_loss, accuracy, _ = test(model, device, test_loader, criterion)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "cifar_cnn.pt")

# Q3. VGG16 Network

VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper `Very Deep Convolutional Networks for Large-Scale Image Recognition`.

The model achieves 92.7% top-5 test accuracy in ImageNet, which is a dataset of over 14 million images belonging to 1000 classes. It was one of the famous model submitted to ILSVRC-2014.

It makes the improvement over AlexNet by replacing large kernel-sized filters (11 and 5 in the first and second convolutional layer, respectively) with multiple 3×3 kernel-sized filters one after another.

![](../assets/vgg16.png)

In [ ]:
from torchvision import models

In [ ]:
model = models.vgg16(pretrained=True).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.05)
criterion = F.cross_entropy

In [ ]:
for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    test_loss, accuracy, _ = test(model, device, test_loader, criterion)
    
    print('Epoch: {}\t Loss: {:.6f}'.format(epoch, train_loss))
    print('\t\t Average Loss: {:.4f}, Accuracy: {:.0f}%'.format(test_loss, accuracy))

torch.save(model.state_dict(), "cifar_cnn.pt")

## Implement VGG model code

Write the model code by looking at the picture of the VGG16 given above.